# Problema de Negócio:

## Quais gêneros de animes tendem a receber notas mais altas?

# Importando Libs

In [440]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

# Carregando Dataset

In [441]:
anime = pd.read_csv("animes.csv")


# Analisando os Dados

In [442]:
anime.shape

(15262, 36)

In [443]:
anime.head()

,Unnamed: 0,Synonyms,Japanese,Type,Episodes,Status,Aired,Premiered,Producers,Licensors,...,Not Recommended,French,Theme,English,Themes,German,Spanish,Genre,Broadcast,Demographics
0,0,Frieren at the Funeral,葬送のフリーレン,TV,28,Finished Airing,"Sep 29, 2023 to Mar 22, 2024",Fall 2023,"Aniplex, Dentsu, Shogakukan-Shueis...","None found, add some",...,42 Not Recommended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,鋼の錬金術師 FULLMETAL ALCHEMIST,TV,64,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",Spring 2009,"Aniplex, Square Enix, Mainichi Bro...","Funimation, Aniplex of America",...,39 Not Recommended,Fullmetal Alchemist Brotherhood,MilitaryMilitary,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,STEINS;GATE,TV,24,Finished Airing,"Apr 6, 2011 to Sep 14, 2011",Spring 2011,"Frontier Works, Media Factory, Kad...",Funimation,...,73 Not Recommended,NaN,NaN,Steins;Gate,"PsychologicalPsychological, Time Trave...",NaN,NaN,NaN,NaN,NaN
3,3,Gintama' (2015),銀魂°,TV,51,Finished Airing,"Apr 8, 2015 to Mar 30, 2016",Spring 2015,"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",...,1 Not Recommended,Gintama Saison 4,NaN,Gintama Season 4,"Gag HumorGag Humor, HistoricalHistoric...",Gintama Season 4,Gintama Temporada 4,NaN,NaN,NaN
4,4,NaN,進撃の巨人 Season3 Part.2,TV,10,Finished Airing,"Apr 29, 2019 to Jul 1, 2019",Spring 2019,"Production I.G, Dentsu, Mainichi B...",Funimation,...,11 Not Recommended,L'Attaque des Titans Saison 3 Partie 2,NaN,Attack on Titan Season 3 Part 2,"GoreGore, MilitaryMilitary, Su...",Attack on Titan Staffel 3 Teil 2,Ataque a los Titanes Temporada 3 Parte 2,NaN,NaN,NaN


In [444]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15262 entries, 0 to 15261
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       15262 non-null  int64 
 1   Synonyms         6453 non-null   object
 2   Japanese         15047 non-null  object
 3   Type             15262 non-null  object
 4   Episodes         15262 non-null  object
 5   Status           15262 non-null  object
 6   Aired            15262 non-null  object
 7   Premiered        5527 non-null   object
 8   Producers        15262 non-null  object
 9   Licensors        15262 non-null  object
 10  Studios          15253 non-null  object
 11  Source           15262 non-null  object
 12  Genres           9232 non-null   object
 13  Demographic      5498 non-null   object
 14  Duration         15262 non-null  object
 15  Rating           15122 non-null  object
 16  Score            15262 non-null  object
 17  Ranked           15262 non-null

### Dectando Valores Duplicados

In [445]:
# Verificar se há linhas duplicadas
anime.duplicated().sum()

0

### Dectando Valores Ausentes

In [446]:
valAusentes = anime.isnull().sum().sort_values(ascending = False)

print(valAusentes)

Demographics       15209
Spanish            13953
German             13945
French             13810
Broadcast          13077
Themes             11323
Genre              10481
Theme               9966
Demographic         9764
Premiered           9735
English             9480
Synonyms            8809
English Name        8422
Genres              6030
Japanese             215
Rating               140
Studios                9
Name                   0
Not Recommended        0
Mixed Feelings         0
Recommended            0
Unnamed: 0             0
description            0
Favorites              0
Members                0
Ranked                 0
Score                  0
Duration               0
Source                 0
Licensors              0
Producers              0
Aired                  0
Status                 0
Episodes               0
Type                   0
Popularity             0
dtype: int64


In [447]:
# Calculando o percentual de valores ausentes
valores_ausentes_percent = valAusentes[valAusentes > 0] / anime.shape[0] 

print(f'{valores_ausentes_percent * 100} %')

Demographics    99.652732
Spanish         91.423142
German          91.370725
French          90.486175
Broadcast       85.683397
Themes          74.190801
Genre           68.673830
Theme           65.299437
Demographic     63.975888
Premiered       63.785873
English         62.115057
Synonyms        57.718517
English Name    55.182807
Genres          39.509894
Japanese         1.408728
Rating           0.917311
Studios          0.058970
dtype: float64 %


# Limpeza e Tratamento dos Dados

> Vamos criar um dataframe novo e deixar apenas as colunas que precisamos

In [448]:
df_anime = anime[['Genres', 'Score', 'Name', 'Genre']]

df_anime.sample(20)

,Genres,Score,Name,Genre
122,"AdventureAdventure, ComedyComedy, ...","8.521 (scored by 218347218,347 users) 1 ...",Sora yori mo Tooi Basho,NaN
6224,"ActionAction, Sci-FiSci-Fi","6.601 (scored by 19001,900 users) 1 ...",Super Robot Taisen OG The Animation,NaN
5935,NaN,"6.651 (scored by 27782,778 users) 1 ...",Perman (1983),ComedyComedy
6939,"AdventureAdventure, FantasyFantasy","6.471 (scored by 13611,361 users) 1 ...",Yoroiden Samurai Troopers Kikoutei Densetsu,NaN
3097,"ComedyComedy, EcchiEcchi","7.221 (scored by 441580441,580 users) 1 ...",Shimoneta to Iu Gainen ga Sonzai Shinai Taikut...,NaN
410,"DramaDrama, FantasyFantasy, Ro...","8.181 (scored by 476192476,192 users) 1 ...",No Game No Life: Zero,NaN
5118,"ActionAction, DramaDrama, Fant...","6.801 (scored by 176931176,931 users) 1 ...",Black★Rock Shooter (TV),NaN
7554,"ComedyComedy, DramaDrama, Spor...",6.361 (scored by 443443 users) 1 ...,Ganbare! Kickers Specials,NaN
14932,NaN,N/A1 (scored by - users) 1 indic...,Xiu Rong Zhen,DramaDrama
1888,NaN,"7.491 (scored by 5122151,221 users) 1 ...",Grappler Baki: Saidai Tournament-hen,SportsSports


> Vamos unificar as colunas de genero

In [449]:
# Juntar as colunas 'Genres' e 'Genre'
df_anime.loc[:, 'Gen'] = df_anime['Genres'].combine_first(df_anime['Genre'])

df_anime.head()

C:\Users\55119\AppData\Local\Temp\ipykernel_14948\959284267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_anime.loc[:, 'Gen'] = df_anime['Genres'].combine_first(df_anime['Genre'])


,Genres,Score,Name,Genre,Gen
0,"AdventureAdventure, DramaDrama, ...","9.361 (scored by 412243412,243 users) 1 ...",Sousou no Frieren,NaN,"AdventureAdventure, DramaDrama, ..."
1,"ActionAction, AdventureAdventure, ...","9.091 (scored by 21271002,127,100 users) ...",Fullmetal Alchemist: Brotherhood,NaN,"ActionAction, AdventureAdventure, ..."
2,"DramaDrama, Sci-FiSci-Fi, Susp...","9.071 (scored by 14042041,404,204 users) ...",Steins;Gate,NaN,"DramaDrama, Sci-FiSci-Fi, Susp..."
3,"ActionAction, ComedyComedy, Sc...","9.061 (scored by 252236252,236 users) 1 ...",Gintama°,NaN,"ActionAction, ComedyComedy, Sc..."
4,"ActionAction, DramaDrama, Susp...","9.051 (scored by 15947041,594,704 users) ...",Shingeki no Kyojin Season 3 Part 2,NaN,"ActionAction, DramaDrama, Susp..."


In [450]:
# Remover as colunas 'Genres' e 'Genre'
df_anime = df_anime.drop(['Genres', 'Genre'], axis=1)

df_anime.head()

,Score,Name,Gen
0,"9.361 (scored by 412243412,243 users) 1 ...",Sousou no Frieren,"AdventureAdventure, DramaDrama, ..."
1,"9.091 (scored by 21271002,127,100 users) ...",Fullmetal Alchemist: Brotherhood,"ActionAction, AdventureAdventure, ..."
2,"9.071 (scored by 14042041,404,204 users) ...",Steins;Gate,"DramaDrama, Sci-FiSci-Fi, Susp..."
3,"9.061 (scored by 252236252,236 users) 1 ...",Gintama°,"ActionAction, ComedyComedy, Sc..."
4,"9.051 (scored by 15947041,594,704 users) ...",Shingeki no Kyojin Season 3 Part 2,"ActionAction, DramaDrama, Susp..."


> Vamos verificar se não tem valores nulos

In [451]:
valAusentes = df_anime.isnull().sum().sort_values(ascending = False)

print(valAusentes)

Gen      1249
Score       0
Name        0
dtype: int64


In [452]:
valores_ausentes_percent = valAusentes[valAusentes > 0] / df_anime.shape[0] 
print(f'{valores_ausentes_percent * 100} %')

Gen    8.183724
dtype: float64 %


> Por termos valores nulos vamos trata-los

In [453]:
# Função para o Replace das linhas vazias com a mediana
def impute_median(dados):
    return dados.fillna(dados.median())

In [454]:
# Removendo valores nulos
#df_anime.loc[:,'Score'] = df_anime['Score'].transform(impute_median)

# Imputando a coluna 'Gen' com a moda (valor mais frequente) usando .loc
mode_gen = df_anime['Gen'].mode()[0]  # obtendo a moda
df_anime.loc[:, 'Gen'] = df_anime['Gen'].fillna(mode_gen)  # preenchendo valores nulos com a moda

In [455]:
valAusentes = df_anime.isnull().sum().sort_values(ascending = False)

print(valAusentes)

Score    0
Name     0
Gen      0
dtype: int64


> Vamos tratar a coluna Score

In [456]:
#Extrair apenas os valores numéricos da coluna 'Score'
df_anime['Score'] = df_anime['Score'].astype(str).str.extract(r'(\d+\.\d+)')[0]

#Convertendo a coluna Score para float, se necessário
df_anime['Score'] = df_anime['Score'].astype(float)

df_anime.head()

,Score,Name,Gen
0,9.361,Sousou no Frieren,"AdventureAdventure, DramaDrama, ..."
1,9.091,Fullmetal Alchemist: Brotherhood,"ActionAction, AdventureAdventure, ..."
2,9.071,Steins;Gate,"DramaDrama, Sci-FiSci-Fi, Susp..."
3,9.061,Gintama°,"ActionAction, ComedyComedy, Sc..."
4,9.051,Shingeki no Kyojin Season 3 Part 2,"ActionAction, DramaDrama, Susp..."


In [457]:
#Verificando valores nulos
valAusentes = df_anime.isnull().sum().sort_values(ascending = False)

print(valAusentes)

Score    1945
Name        0
Gen         0
dtype: int64


In [458]:
# Removendo valores nulos
df_anime.loc[:,'Score'] = df_anime['Score'].transform(impute_median)

In [459]:
valAusentes = df_anime.isnull().sum().sort_values(ascending = False)

print(valAusentes)

Score    0
Name     0
Gen      0
dtype: int64


> Vamos tratar a coluna Gen

In [470]:
# Função para tratar entradas na coluna 'Gen'
def clean_genres(genres):
    # Remove duplicatas que aparecem escritas juntas (ex: Sci-FiSci-Fi)
    cleaned_genres = re.sub(r'(\b\w+\b)\1+', r'\1', genres)  # Remove duplicatas concatenadas

    # Divide a string em partes, remove espaços e mantém únicos
    genre_list = [genre.strip() for genre in cleaned_genres.split(',')]
    
    # Remove duplicatas e preserva a ordem
    unique_genres = []
    for genre in genre_list:
        # Verifica se o gênero já está na lista e não considera espaços extras
        if genre.lower() not in (g.lower() for g in unique_genres):
            unique_genres.append(genre)
    
    # Junta novamente em uma string
    return ', '.join(unique_genres)

# Aplicar a função para limpar a coluna 'Gen'
df_anime['Gen'] = df_anime['Gen'].apply(clean_genres)

# Verificar as alterações
df_anime.head(50)


,Score,Name,Gen
0,9.361,Sousou no Frieren,"Adventure, Drama, Fantasy"
1,9.091,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy"
2,9.071,Steins;Gate,"Drama, Sci-FiSci-Fi, Suspense"
3,9.061,Gintama°,"Action, Comedy, Sci-FiSci-Fi"
4,9.051,Shingeki no Kyojin Season 3 Part 2,"Action, Drama, Suspense"
5,9.041,Gintama: The Final,"Action, Comedy, Drama, Sci-FiSci-Fi"
6,9.031,Gintama',"Action, Comedy, Sci-FiSci-Fi"
7,9.031,Hunter x Hunter (2011),"Action, Adventure, Fantasy"
8,9.021,Ginga Eiyuu Densetsu,"Drama, Sci-FiSci-Fi"
9,9.021,Gintama': Enchousen,"Action, Comedy, Sci-FiSci-Fi"
